## Stage 2 of Image Processing Pipeline

All the chunks for a patient are processed to produce embeddings for each chunk. This is also repeated with an offset of W/2 (12). All the embeddings produced are concatenated and passed to a transformer encoder and linear layer to predict the probability of PE for a patient.

In [3]:
from sequence_dataset import ChunkDataset
from sequence_model import CTPAModel
from test_model import do_test
from torch.utils.data import Dataset, DataLoader

In [1]:
import pickle
import pandas as pd
#reading embeddings with and without offset
file_paths0 = [ "C:\\Users\\preet\\Documents\\penet\\results\\best_result_shift_0\\embs_train.pickle",
              "C:\\Users\\preet\\Documents\\penet\\results\\best_result_shift_0\\embs_val.pickle",
              "C:\\Users\\preet\\Documents\\penet\\results\\best_result_shift_0\\embs_test.pickle"]

file_paths1 = [ "C:\\Users\\preet\\Documents\\penet\\results\\best_result_shift_1\\embs_train.pickle",
              "C:\\Users\\preet\\Documents\\penet\\results\\best_result_shift_1\\embs_val.pickle",
              "C:\\Users\\preet\\Documents\\penet\\results\\best_result_shift_1\\embs_test.pickle"]

def return_loaded_files(file_paths):
  
    with open(file_paths[0], 'rb') as f:
            # Load the pickled object from the file
            loaded_data_train = pickle.load(f)

    with open(file_paths[1], 'rb') as f:
            # Load the pickled object from the file
            loaded_data_val = pickle.load(f)
  
    with open(file_paths[2], 'rb') as f:
            # Load the pickled object from the file
            loaded_data_test = pickle.load(f)
    return loaded_data_train,loaded_data_val,loaded_data_test


In [2]:
#Create one dataframe with the embeddings with and without offsent for each slice for a patient as well as the probability of PE for each slice


emb_train, emb_val, emb_test = return_loaded_files(file_paths0)
emb_train1, emb_val1, emb_test1 = return_loaded_files(file_paths1)
print(emb_train.keys())
print(len(emb_train['patient']))
print(len(emb_train1['patient']))
print(emb_train['emb'][0].shape)

df_train = pd.DataFrame(emb_train)
df_train["emb1"] = emb_train1["emb"]
df_train["prob1"] = emb_train1["prob"]
df_val = pd.DataFrame(emb_val)
df_val["emb1"] = emb_val1["emb"]
df_val["prob1"] = emb_val1["prob"]

df_test = pd.DataFrame(emb_test)
df_test["emb1"] = emb_test1["emb"]
df_test["prob1"] = emb_test1["prob"]

df_test.head(20)

dict_keys(['patient', 'slice', 'prob', 'label', 'emb'])
24128
24128
(384,)


,patient,slice,prob,label,emb,emb1,prob1
0,2506,0,0.049225,0,"[1.6442688, 0.25198266, 0.53300434, -0.4185633...","[1.6442688, 0.25198266, 0.53300434, -0.4185633...",0.049225
1,2506,1,0.052704,0,"[1.691245, 0.56891054, 0.23200284, -0.73625606...","[1.5159245, 0.33395997, 0.46993455, -0.5660441...",0.050232
2,2506,2,0.095337,0,"[1.7385237, 0.6826804, -0.18666513, -0.8393175...","[1.8511521, 0.68758553, -0.061858013, -0.73435...",0.067566
3,2506,3,0.194336,0,"[1.5697328, 0.69841456, -0.011741816, -1.05351...","[1.66374, 0.65868104, -0.095052205, -0.9741797...",0.147339
4,2506,4,0.207275,0,"[1.6401, 0.64765483, -0.092930436, -1.0762582,...","[1.5658956, 0.70841485, -0.01152911, -1.120370...",0.214478
5,2506,5,0.370117,0,"[1.3882405, 0.8310233, -0.38720456, -1.4329963...","[1.6777356, 0.668254, -0.110921256, -1.15693, ...",0.220825
6,2506,6,0.269531,0,"[1.6847997, 0.32295343, -0.3549741, -1.1599443...","[1.2737544, 0.6220103, -0.67503995, -1.2819692...",0.427246
7,2506,7,0.255615,0,"[1.720989, 0.5370865, -0.24712141, -1.2112033,...","[1.7898481, 0.38958457, -0.25716192, -1.102011...",0.226807
8,2506,8,0.312988,0,"[1.4537445, 0.7034186, -0.3505595, -1.3398671,...","[1.6212622, 0.62308013, -0.27560562, -1.242563...",0.271729
9,2506,9,0.221680,0,"[1.5071402, 0.71281886, -0.03806496, -1.154002...","[1.3956892, 0.7283744, -0.29031906, -1.3580737...",0.316895


In [ ]:
trn_dataset = ChunkDataset(df_train)
val_dataset = ChunkDataset(df_val)
test_dataset = ChunkDataset(df_test)
train_dataloader = DataLoader(trn_dataset, batch_size=4, num_workers=0,shuffle=True,drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, num_workers=0,drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, num_workers=0,drop_last=True)

In [141]:
embed_dim = 384
num_heads = 8
num_layers = 4

model = CTPAModel(embed_dim, num_heads, num_layers, drop_prob=0.1)
# Alternatively, to just get the total parameter count:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params/1e6}")

Total parameters: 11.23533


## Train and Test Model

In [143]:
from tqdm import tqdm
import torch.optim as optim
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score,balanced_accuracy_score, precision_score, recall_score
import torch
from sklearn.metrics import roc_curve, roc_auc_score, RocCurveDisplay
import os
from sklearn.metrics import average_precision_score
criterion = torch.nn.CrossEntropyLoss()

# Define the optimizer (Adam)
learning_rate = 1e-5
num_epochs = 5
model = CTPAModel(embed_dim, num_heads, num_layers,drop_prob=0.2)

model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
from torch.optim.lr_scheduler import CosineAnnealingLR   # or CosineAnnealingWarmRestarts

min_lr   = 1e-6                      # floor value
scheduler = CosineAnnealingLR(
    optimizer,
    T_max=num_epochs,                # # of scheduler updates before it restarts (here: 1 cycle = whole run)
    eta_min=min_lr                   # lowest LR the cosine will hit
)
use_amp = True
device = "cuda"
thresh = 0
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
train_losses = []
val_losses = []
val_rocs=[]
test_losses = []
test_rocs=[]
best = 0
for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        total_accuracy = 0.0
        num_train_batches = len(train_dataloader)
        
        all_preds = []
        all_labels = []
        f1_scores =[]
        
        for num_batch,batch in enumerate(tqdm(train_dataloader, total=num_train_batches)):
            if num_batch < num_train_batches:
                with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
                    inputs = batch[0] #.to(device)
                    
                    inputs = inputs.to(device)
                    labels = torch.squeeze(batch[1].to(device))
                   # print(labels)
                    #M = (torch.rand(inputs.size()) > thresh).to(torch.long)
                    #M = M.cuda()
                    #print(M)
                    #One = torch.ones(inputs.size()).cuda()
                    #inputs = M * inputs + (One-M)*(One-inputs)
                    outputs,_ = model(inputs) #.to(torch.long))
                  #  print(outputs)
    
                   # print(outputs.size(), labels.size())
                    #print(outputs.shape, labels.shape)
                    loss = criterion(outputs, labels)
                    
                    preds = torch.argmax(outputs, dim=1) 
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scheduler.step()
                scaler.update()
                
                optimizer.zero_grad(set_to_none=True) # set_to_none=True here can modestly improve performance
                total_loss += loss.item()
            #if num_batch == 100:
            #    end_timer_and_print("Test")
                
       # end_timer_and_print("Test")

        avg_train_loss = total_loss / num_train_batches
        train_losses.append(avg_train_loss)
        f1 = f1_score(all_labels, all_preds, average='macro')
        acc = accuracy_score(all_labels, all_preds)
        ba = balanced_accuracy_score(all_labels, all_preds)
      
        precision = precision_score(all_labels, all_preds)
        recall = recall_score(all_labels, all_preds)
        f1_scores.append(f1)
        confusion_matrix_sc = confusion_matrix(all_labels, all_preds)
        print(f'Epoch {epoch + 1}')
        print("Train loss{b:.3f}, F1 {c:.3f}, Acc {d:.3f}, BA {e:.3f}, precision {f:.3f}, recall {g:.3f}".format( b=avg_train_loss, c=f1, d = acc, e= ba, f=precision, g=recall))
        print(f"cm{confusion_matrix_sc}") 
        model.eval()
        
        #x = do_test(val_dataloader)
        #val_losses.append(x.item())
        save_dir = r"C:\Users\preet\Documents\penet\results_sequence"
        os.makedirs(save_dir, exist_ok=True)
        
        # Evaluation
        metrics, output = do_test(val_dataloader, model)
        roc_auc = metrics["AUC-ROC"]
        
        # Save if best
        if roc_auc >= best:
            best = roc_auc
            print("Val AUC-ROC:", roc_auc)
            val_rocs.append(roc_auc)
            fname = os.path.join(save_dir, "best_model.pth")
            torch.save(model.state_dict(), fname)

C:\Users\preet\AppData\Local\Temp\ipykernel_21304\4024281372.py:28: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
100%|████████████████████████████████████████████████████████████████████████████████| 357/357 [00:13<00:00, 27.31it/s]


Epoch 1
Train loss0.486, F1 0.700, Acc 0.767, BA 0.686, precision 0.773, recall 0.438
cm[[884  62]
 [271 211]]


C:\Users\preet\AppData\Local\Temp\ipykernel_21304\2734414312.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output.extend(F.softmax(outputs).cpu().numpy()[:, 1])
100%|█████████████████████████████████████████████████████████████████████████████████| 82/82 [00:00<00:00, 107.69it/s]


Test loss0.471, F1 0.720, Acc 0.799, BA 0.794, precision 0.449, recall 0.786
cm[[109  27]
 [  6  22]]
Val AUC-ROC: 0.8534663865546218


100%|████████████████████████████████████████████████████████████████████████████████| 357/357 [00:12<00:00, 28.14it/s]


...

Test loss0.570, F1 0.707, Acc 0.780, BA 0.797, precision 0.426, recall 0.821
cm[[105  31]
 [  5  23]]
Val AUC-ROC: 0.8555672268907564


100%|████████████████████████████████████████████████████████████████████████████████| 357/357 [00:12<00:00, 28.12it/s]


Epoch 5
Train loss0.267, F1 0.896, Acc 0.910, BA 0.884, precision 0.919, recall 0.803
cm[[911  34]
 [ 95 388]]


C:\Users\preet\AppData\Local\Temp\ipykernel_21304\2734414312.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output.extend(F.softmax(outputs).cpu().numpy()[:, 1])
100%|█████████████████████████████████████████████████████████████████████████████████| 82/82 [00:00<00:00, 101.82it/s]


Test loss0.428, F1 0.784, Acc 0.860, BA 0.830, precision 0.564, recall 0.786
cm[[119  17]
 [  6  22]]
Val AUC-ROC: 0.8568802521008404


In [150]:
def create_metrics(result_df):
    from sklearn.metrics import (
            accuracy_score, precision_score, recall_score, f1_score,
            roc_auc_score, average_precision_score, balanced_accuracy_score,
            confusion_matrix
        )
    import pandas as pd
    
    # Get true labels and predicted probabilities
    y_true = result_df['label'].values
    y_prob = result_df['final_prob'].values
    y_pred = (y_prob >= 0.5).astype(int)
    
    # Confusion matrix to get TN, FP, FN, TP
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # Compute metrics
    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Sensitivity": recall_score(y_true, y_pred),  # aka recall
        "Specificity": tn / (tn + fp) if (tn + fp) > 0 else 0.0,
        "Precision": precision_score(y_true, y_pred),
        "F1 Score": f1_score(y_true, y_pred),
        "AUC-ROC": roc_auc_score(y_true, y_prob),
        "AUC-PR": average_precision_score(y_true, y_prob),
        "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred)
    }
        
    # Add to metrics_df
    metrics_df2 = pd.DataFrame([metrics])  # single row
    return metrics_df2 

